In [ ]:
# LDA
# Change the NAME.csv

import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # Importar numpy para usar unique()
from sklearn.preprocessing import LabelEncoder # Importar LabelEncoder

# Carregar o arquivo CSV
# A primeira coluna (index 0) é a classe, as colunas subsequentes são os espectros (número de onda)
df = pd.read_csv('NAME.csv')

# Separar as features (X) e o target (y)
# Ignoramos a primeira linha que contém o número de onda, assumindo que o pandas lê corretamente as colunas.
# A primeira coluna é a classe.
X = df.iloc[:, 1:]  # Todas as colunas exceto a primeira
y = df.iloc[:, 0]   # A primeira coluna é a classe

# Converter os rótulos de classe string para numéricos usando Label Encoding
# Isso é necessário para que o Matplotlib possa mapear as classes para cores no scatter plot
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42) # Usar y_encoded aqui para o treino

# Inicializar e treinar o modelo LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = lda.predict(X_test)

# Avaliar o modelo
# É importante usar os rótulos originais (ou decodificados) para o classification_report e confusion_matrix
# Para o classification_report e confusion_matrix, precisamos dos rótulos originais ou decodificados
# Vamos decodificar y_test e y_pred para a avaliação
y_test_decoded = label_encoder.inverse_transform(y_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

print("Classification Report:")
print(classification_report(y_test_decoded, y_pred_decoded))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test_decoded, y_pred_decoded, labels=label_encoder.classes_) # Especificar os rótulos para garantir a ordem correta
print(cm)

# Visualizar a matriz de confusão
plt.figure(figsize=(8, 6))
# Usar os rótulos decodificados para os ticklabels
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Visualizar os resultados do LDA (se houver mais de uma dimensão discriminante)
# LDA pode reduzir a dimensionalidade para um máximo de n_classes - 1 dimensões.

# Transformar os dados para a espaço LDA
X_lda = lda.transform(X) # Transformar os dados originais X, não os de treino ou teste

# Obter o número de componentes discriminantes
# O número de componentes é min(n_classes - 1, n_features).
# Como estamos plotando, geralmente nos preocupamos com as primeiras componentes.
# Podemos inferir o número de componentes a partir da forma de X_lda ou do número de classes.
n_components = X_lda.shape[1] # O número de colunas em X_lda é o número de componentes

# Alternativamente, você pode contar o número de classes únicas
n_classes = len(label_encoder.classes_) # Usar o número de classes únicas do label encoder
# n_components = min(n_classes - 1, X.shape[1])


if n_components > 1:
    plt.figure(figsize=(10, 8))
    # Usar X_lda[:, 0] e X_lda[:, 1] para as duas primeiras componentes
    # Usar y_encoded para a colorização
    scatter = plt.scatter(X_lda[:, 0], X_lda[:, 1], c=y_encoded, cmap='viridis', edgecolors='k', s=50)
    plt.xlabel('LD1')
    plt.ylabel('LD2')
    plt.title('LDA of FTIR Spectra')
    # Criar uma colorbar com os rótulos originais
    cbar = plt.colorbar(scatter)
    cbar.set_label('Class')
    # Definir os ticks e labels da colorbar
    tick_locs = (y_encoded.min() + y_encoded.max()) / (2 * n_classes) + np.arange(n_classes) * (y_encoded.max() - y_encoded.min() + 1) / n_classes
    cbar.set_ticks(tick_locs)
    cbar.set_ticklabels(label_encoder.classes_)
    plt.show()
elif n_components == 1:
    plt.figure(figsize=(10, 6))
    # Usar X_lda[:, 0] para a primeira componente
    # Usar y_encoded para a colorização
    sns.histplot(x=X_lda[:, 0], hue=y_encoded, multiple="stack", kde=True, palette='viridis')
    plt.xlabel('LD1')
    plt.title('LDA of FTIR Spectra (1D)')
    # Para o histplot, a legenda é gerada automaticamente com os valores numéricos.
    # Poderíamos tentar customizar a legenda se necessário, mas para uma simples visualização
    # com 1D, os números podem ser aceitáveis ou pode-se adicionar uma anotação.
    plt.show()
else:
    print("LDA did not produce any discriminant components.")


# Se você quiser usar o modelo treinado para prever novas amostras:
# Lembre-se de que o modelo foi treinado com rótulos numéricos.
# Se você prever uma nova amostra, o resultado será um número.
# Você precisará usar label_encoder.inverse_transform() para obter o rótulo original.
# new_spectrum = [seu_novo_espectro_aqui]
# predicted_class_encoded = lda.predict([new_spectrum])
# predicted_class_decoded = label_encoder.inverse_transform(predicted_class_encoded)
# print(f"Predicted class for new spectrum: {predicted_class_decoded[0]}")

In [ ]:
# Top 3 wavenumbers with highest absolute weights in LD1

import numpy as np
# Para identificar os pesos das features (números de onda), podemos analisar os coeficientes
# do modelo LDA. Os coeficientes indicam a importância de cada feature na combinação linear
# que forma as componentes discriminantes.

# Obter os coeficientes do modelo LDA
# Para o LDA, os coeficientes estão em `lda.coef_`.
# A forma é (n_classes - 1, n_features) ou (n_components, n_features)
# onde n_components é o número de componentes discriminantes geradas.
lda_coefficients = lda.coef_

# Os números de onda correspondem aos nomes das colunas em X.
# A primeira linha do arquivo original (que foi ignorada pelo pandas) contém os números de onda.
# Precisamos carregar essa linha separadamente ou assumir que os nomes das colunas do DataFrame X
# correspondem aos números de onda se o CSV tiver cabeçalho.
# Se o CSV não tem cabeçalho, precisaremos ler a primeira linha manualmente.

# Assumindo que a primeira linha do arquivo original contém os números de onda
# Vamos reler a primeira linha do arquivo CSV para obter os nomes das colunas
with open('AMP_preprocess.csv', 'r') as f:
    first_line = f.readline().strip()

# A primeira linha contém a classe e depois os números de onda separados por vírgula
# Separar os números de onda (ignorando o primeiro elemento que é o nome da coluna da classe)
wavenumbers = first_line.split(',')[1:]

# Converter os números de onda para tipo numérico (float, por exemplo)
# Isso garante que a ordenação seja numérica e não lexicográfica
wavenumbers = [float(wn) for wn in wavenumbers]


# Para cada componente discriminante, podemos encontrar as features com os maiores pesos (em magnitude)
# Se houver apenas uma componente discriminante (n_components == 1), analisamos os pesos dessa componente.
# Se houver múltiplas componentes, podemos analisar a primeira componente ou a combinação delas.
# Geralmente, a primeira componente discriminante (LD1) explica a maior parte da variância entre as classes.
# Vamos analisar os pesos da primeira componente discriminante (lda_coefficients[0]).

if n_components >= 1:
    # Os pesos da primeira componente discriminante
    weights_ld1 = lda_coefficients[0]

    # Obter os índices das features com os maiores pesos em magnitude (valor absoluto)
    # Usar np.argsort para obter os índices que ordenariam os pesos por magnitude
    # Pegar os últimos 3 índices para obter os 3 maiores pesos em magnitude
    top_3_indices = np.argsort(np.abs(weights_ld1))[-3:]

    # Ordenar os índices para obter os pesos em ordem decrescente de magnitude
    top_3_indices = top_3_indices[::-1]

    print("\nTop 3 wavenumbers with highest absolute weights in LD1:")
    for i in top_3_indices:
        # Usar os índices para obter o número de onda e o peso correspondente
        wavenumber = wavenumbers[i]
        weight = weights_ld1[i]
        print(f"Wavenumber: {wavenumber}, Weight (LD1): {weight:.4f}")

    # Se houver mais de uma componente, você pode repetir a análise para outras componentes
    # ou considerar uma combinação dos pesos. Para simplificar, focamos na LD1 que é a mais importante.

else:
    print("\nLDA did not produce any discriminant components to analyze weights.")

